# Landing Performance Assignment

In [77]:
import math
import qty
import numpy as np
import atmosphere

## Deliverable 1 - Dispatch Landing Chart

### Conditions:
- Dry AND wet runways
- Mass variation from 170 to 230 tons, steps of 10 tons
- Field elevation sea level, up to and including 5000 ft, steps of 1000 ft
- Zero wind

### Aircraft data
- Wingspan, $b = 64.5 m$
- Wing surface, $S = 436.8 m^2$
- Oswald factor, $e = 0.8$
- Parasitic drag coefficient in landing configuration, $C_{D_0} = 0.088$
- Braking coefficient on dry runway, $\mu_{dry} = 0.54$
- Braking coefficient on wet runway, $\mu_{wet} = 0.34$
- Lift coefficient in ground run, $C_{L_{gnd}} = 0.126$
- Drag coefficient in ground run, $C_{D_{gnd}} = 0.213$
- IAS stall at 200 tons with flaps 30, ${IAS}_{stall} = 122 Kts$
- Speed during approach, $v_{ref} = 1.23 \cdot {IAS}_{stall}$
- Speed at touchdown, $v_{td} = 1.18 \cdot {IAS}_{stall}$
- Time of transition phase, $t_{tr} = 1.3 s$
- Speed decay factor of transition phase of landing run, $decay = 0.85$
- Thrust per engine at touchdown, $T = 3400 lbs$

### Assumptions
- Landing flaps 30
- Drag polar is exactly parabolic

Initialize the variables in python:

In [78]:
mass = qty.Mass.M_ton(200)
wingspan = qty.Distance(64.5)
surface = qty.Area(436.8)
oswald = 0.8
cd0_land = 0.088
mu_dry = 0.54
mu_wet = 0.34
cl_gnd = 0.126
cd_gnd = 0.213
ias_stall = qty.Velocity.Kts(122)
v_stall_to_ref = 1.23
v_stall_to_td = 1.18
transition_time = qty.Time(1.3)
speed_decay = 0.85
thrust_land = qty.Force.Lbf(3400 * 2)


# More assumptions

# Modern aircraft, so flare height = screen height = 50 ft
flare_height = qty.Distance.Ft(50)

# Assume that the flare is decelerating

# Assume ILS approach, 3 degrees fpa
fpa = math.radians(3)

In [79]:
aspect_ratio = wingspan**2 / surface
cdi_k_const = 1 / (math.pi * oswald * aspect_ratio)
cdi_k_const

0.04177566200501165

In [80]:
cl_max = (2 * 9.81 * mass) / (1.225 * ias_stall**2 * surface)
print(cl_max)

1.8617169631927402


In [81]:
def get_tas_stall(mass, elevation):
    density = atmosphere.get_density_at_height(elevation)
    ias_stall = math.sqrt((2 * 9.81 * mass) / (surface * cl_max * density))
    tas_stall = ias_stall * math.sqrt(1.225 / density)
    
    return tas_stall

In [82]:
def average_tmind(mass, elevation, tas_stall):
    weight = mass * 9.81
    t_min_d_b = -(weight * math.sin(fpa))
    density = atmosphere.get_density_at_height(elevation)
    tas_td = tas_stall * v_stall_to_td
    drag_td = qty.Force(0.5 * density * tas_td**2 * surface * cd_gnd)
    t_min_d_c = qty.Force(thrust_land - drag_td)
    t_min_d_avg = qty.Force((t_min_d_b + t_min_d_c) / 2)
    
    return t_min_d_avg

In [94]:
masses = [qty.Mass.M_ton(m) for m in np.arange(170, 240, 10)]
elevations = [qty.Distance.Ft(h) for h in np.arange(0, 6000, 1000)]

for mass1 in masses:
    for elev in elevations:
        tas_stall = get_tas_stall(mass1, elev)
        t_min_d_avg = average_tmind(mass1, elev, tas_stall)
        
        weight = mass1 * 9.81
        density_flare = atmosphere.get_density_at_height(elev + flare_height)
        density_td = atmosphere.get_density_at_height(elev)
        
        tas_b = tas_stall * v_stall_to_ref
        tas_c = tas_stall * v_stall_to_td
        
        dist_flare = qty.Distance((weight / (2 * 9.81) * (tas_c**2 - tas_b**2) - weight * flare_height) / t_min_d_avg)
        print(f"Mass: {int(mass1.m_ton)} ton, Elevation: {int(elev.ft):04} ft, Flare Distance: {dist_flare}")
    print()

Mass: 170 ton, Elevation: 0000 ft, Flare Distance: 370.04 meters
Mass: 170 ton, Elevation: 1000 ft, Flare Distance: 373.72 meters
Mass: 170 ton, Elevation: 2000 ft, Flare Distance: 377.75 meters
Mass: 170 ton, Elevation: 3000 ft, Flare Distance: 382.12 meters
Mass: 170 ton, Elevation: 4000 ft, Flare Distance: 386.85 meters
Mass: 170 ton, Elevation: 5000 ft, Flare Distance: 391.96 meters

Mass: 180 ton, Elevation: 0000 ft, Flare Distance: 380.55 meters
Mass: 180 ton, Elevation: 1000 ft, Flare Distance: 384.71 meters
Mass: 180 ton, Elevation: 2000 ft, Flare Distance: 389.21 meters
Mass: 180 ton, Elevation: 3000 ft, Flare Distance: 394.08 meters
Mass: 180 ton, Elevation: 4000 ft, Flare Distance: 399.32 meters
Mass: 180 ton, Elevation: 5000 ft, Flare Distance: 404.97 meters

Mass: 190 ton, Elevation: 0000 ft, Flare Distance: 391.18 meters
Mass: 190 ton, Elevation: 1000 ft, Flare Distance: 395.79 meters
Mass: 190 ton, Elevation: 2000 ft, Flare Distance: 400.77 meters
Mass: 190 ton, Elevatio